# Usage of ChatGPT to query different things
add some libraries which are required for this demo

In [40]:
%pip install wikipedia numexpr pypdf faiss-cpu tiktoken

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 24.0
[notice] To update, run: C:\Users\SylvioHellmann\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# load LangChain

In [41]:
import os
import sys

#import openai
#from langchain.chains import ConversationalRetrievalChain, RetrievalQA
#from langchain.chat_models import ChatOpenAI
#from langchain.document_loaders import DirectoryLoader, TextLoader
#from langchain.embeddings import OpenAIEmbeddings
#from langchain.indexes import VectorstoreIndexCreator
#from langchain.indexes.vectorstore import VectorStoreIndexWrapper
#from langchain.llms import OpenAI
#from langchain.memory import ConversationBufferWindowMemory
#from langchain.vectorstores import Chroma

## Set OPENAI API key

You will find your key [here](https://platform.openai.com/account/api-keys)

Copy the key from the portal into the file constants.py

APIKEY = "sk-0e7Q...."

In [42]:
import constants
os.environ["OPENAI_API_KEY"] = constants.APIKEY

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0) # --> it's no longer supported

response = llm.invoke("List the seven wonders of the world with the distance from Munich in Germany. Format the output as table in a markdown language.")
print(response.content)

## overall question
use special sources from the internet e. g. Wikipedia

In [ ]:
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
tools = load_tools(['wikipedia', 'llm-math'], llm=llm)

agent = initialize_agent(
    tools=tools, 
    llm=llm, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    #return_intermediate_steps=True,
    verbose=True,
)

from langchain.callbacks import StdOutCallbackHandler

question = """
What is the current American president's age? 
Return the age multiplied with the number of year as president.
"""

handler = StdOutCallbackHandler()
response = agent(
    {"input": question}, 
    callbacks=[handler]
)

## query PDF file(s)
https://python.langchain.com/v0.2/docs/how_to/document_loader_pdf/

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

loader = PyPDFLoader("data/layout-parser-paper.pdf")
pages = loader.load_and_split()

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]))
docs = faiss_index.similarity_search("What is LayoutParser?", k=2) # What is LayoutParser?, How will the community be engaged?
print(docs)

for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

### query multiple PDF files from a Web App


## question based on CSV file

In [ ]:
import pandas
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.agents import create_csv_agent

agent = create_csv_agent(
    OpenAI(model="gpt-3.5-turbo-instruct", temperature=0), # model="gpt-3.5-turbo-0613"
    "data/test.csv",
    verbose = True,
    # agent_type=AgentType.OPENAI_FUNCTIONS, # AgentType.OPENAI_FUNCTIONS, AgentType.ZERO_SHOT_REACT_DESCRIPTION
)
agent.run("How many items were bought by Latasha Alonso?")

In [ ]:
agent.run("Who bought the most items?")

### query multiple CSV files

https://medium.com/@shweta-lodha/query-multiple-csv-files-together-azure-openai-langchain-0aa2a2b4665f

## usage of an external API

In [ ]:
# using tools e.g. Open Weather API
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType, load_tools

llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

os.environ["OPENWEATHERMAP_API_KEY"] = constants.OPENWEATHERMAP_API_KEY
tools = load_tools(['openweathermap-api'])

agent = initialize_agent(tools, llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)

agent.run("What is the current weather in Munich, Germany?")